# ***`Libraries`***

In [8]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForQuestionAnswering, AdamW
from sklearn.model_selection import train_test_split
import wandb
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
import pandas as pd
import re
from transformers import MobileBertTokenizer, MobileBertForQuestionAnswering, AdamW
import logging
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForQuestionAnswering, AdamW
from sklearn.model_selection import train_test_split
import pandas as pd
from nltk.translate.bleu_score import corpus_bleu
from fractions import Fraction
import logging
from simpletransformers.question_answering import QuestionAnsweringModel



# Load Data set

In [10]:
df = pd.read_csv("train.csv")

# Preprocessing

In [12]:

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text
train_examples = []
for _, row in train_data.iterrows():
    train_examples.append({
        'context': row['Answer'],
        'qas': [{
            'id': row['qtype'],
            'question': row['Question'],
            'answers': [{
                'text': row['Answer'],
                'answer_start': 0
            }]
        }]
    })
test_examples = []
for _, row in test_data.iterrows():
    test_examples.append({
        'context': row['Answer'],
        'qas': [{
            'id': str(row.name),
            'question': row['Question'],
            'answers': [{
                'text': row['Answer'],
                'answer_start': 0
            }]
        }]
    })

# Cleaned Text

In [13]:

df['Question'] = df['Question'].apply(preprocess_text)
df['Answer'] = df['Answer'].apply(preprocess_text)
df

,qtype,Question,Answer
0,susceptibility,who is at risk for lymphocytic choriomeningiti...,lcmv infections can occur after exposure to fr...
1,symptoms,what are the symptoms of lymphocytic choriomen...,lcmv is most commonly recognized as causing ne...
2,susceptibility,who is at risk for lymphocytic choriomeningiti...,individuals of all ages who come into contact ...
3,exams and tests,how to diagnose lymphocytic choriomeningitis lcm,during the first phase of the disease the most...
4,treatment,what are the treatments for lymphocytic chorio...,aseptic meningitis encephalitis or meningoence...
...,...,...,...
95,exams and tests,how to diagnose parasites leishmaniasis,various laboratory methods can be used to diag...
96,treatment,what are the treatments for parasites leishman...,before considering treatment the first step is...
97,prevention,how to prevent parasites leishmaniasis,no vaccines or drugs to prevent infection are ...
98,information,what is are parasites toxoplasmosis toxoplasma...,a singlecelled parasite called toxoplasma gond...


# Aplit the Data

In [14]:
train_data, test_data = train_test_split(df, test_size=0.25, random_state=42)

 # ***`-----------Bert-------------------`***
 # -----------------Bert---------------------
 # `--------------------------------------`

# Gineral Model ARguments

In [ ]:
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.n_best_size = 5
model_args.num_train_epochs = 1
model_args.overwrite_output_dir = True
model_args.save_model_every_epoch = False
model_args.use_multiprocessing = False
model_args.wandb_project = "MedQuad-QA"
model_args.config = {
    "hidden_dropout_prob": 0.3,
    "attention_probs_dropout_prob": 0.3
}
model_args_7 = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.n_best_size = 5
model_args.num_train_epochs = 1
model_args.overwrite_output_dir = True
model_args.save_model_every_epoch = False
model_args.use_multiprocessing = False
model_args.wandb_project = "MedQuad-QA"
model_args.config = {
    "hidden_dropout_prob": 0.7,
    "attention_probs_dropout_prob": 0.7
}
model_args.num_samples = 10  # Set the desired number of samples
logging.basicConfig(level=logging.ERROR)

# Load the Bert

In [ ]:
bert_model_03_dropout = QuestionAnsweringModel(
    "bert", "bert-base-cased", args=model_args, use_cuda=torch.cuda.is_available()
)
bert_model_07_dropout = QuestionAnsweringModel(
    "bert", "bert-base-cased", args=model_args_7,use_cuda=torch.cuda.is_available()
)

c:\Users\mahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# bert_model_03_dropout
5cb8eccab60f9a55795c159a6f083bd3089b708d

In [ ]:
logging.basicConfig(level=logging.ERROR)
try:
    bert_model_03_dropout = QuestionAnsweringModel("bert", "bert-base-cased", args=model_args, use_cuda=torch.cuda.is_available())
    bert_model_03_dropout.train_model(train_examples, eval_data=test_examples, num_samples=5, output_dir="model_output/")
except Exception as e:
    print(f"An error occurred during training: {str(e)}")
try:
    bert_result, _ = bert_model_03_dropout.eval_model(test_examples, output_dir="model_output/", verbose=False)
    print("BERT Results:", bert_result)
    predictions = bert_model_03_dropout.predict(test_examples)
    references = [example['answers'][0]['text'] for example in test_examples]
except Exception as e:
    print(f"An error occurred during evaluation: {str(e)}")


c:\Users\mahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1 of 1:   0%|          | 0/1 [00:10<?, ?it/s]c:\Users\mahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\wandb\sdk\wandb_run.py:2265: UserWarning: Run (6ob2cuvl) is finished. The call to `_console_raw_callback` will be ignored. Please make sure that you are using an active run.
  lambda data: self._console_raw_callback("stderr", data),
Epoch 1 of 1:   0%|          | 0/1 [03:23<?, ?it/s]


An error occurred during training: 'int' object is not callable


Running Evaluation: 100%|██████████| 1/1 [00:10<00:00, 10.88s/it]


BERT Results: {'correct': 0, 'similar': 25, 'incorrect': 0, 'eval_loss': -0.006273320410400629}


Running Prediction: 100%|██████████| 1/1 [00:10<00:00, 10.39s/it]

An error occurred during evaluation: 'answers'


# bert_model_07_dropout

In [ ]:
logging.basicConfig(level=logging.ERROR)
try:
    bert_model_07_dropout = QuestionAnsweringModel("bert", "bert-base-cased", args=model_args_7, use_cuda=torch.cuda.is_available())
    bert_model_07_dropout.train_model(train_examples, eval_data=test_examples, num_samples=5, output_dir="model_output/")
except Exception as e:
    print(f"An error occurred during training: {str(e)}")
try:
    bert_result, _ = bert_model_07_dropout.eval_model(test_examples, output_dir="model_output/", verbose=False)
    print("BERT Results:", bert_result)
    predictions_07 = bert_model_07_dropout.predict(test_examples)
    extracted_predictions = []
    for pred in predictions:
        if 'answer' in pred:
            extracted_predictions.append(pred['answer'][0])
        else:
            extracted_predictions.append("")
    references_07 = [example['context'] for example in test_examples]
except Exception as e:
    print(f"An error occurred during evaluation: {str(e)}")

# Blue Score Bert Model

In [ ]:
min_len = min(len(predictions), len(references))
predictions = predictions[:min_len]
references = references[:min_len]
extracted_predictions = [pred['answer'][0] if isinstance(pred, dict) else pred for pred in predictions]
numerator = sum(min(len(pred), len(ref)) for pred, ref in zip(extracted_predictions, references))
denominator = sum(len(pred) for pred in extracted_predictions)
bleu_score = Fraction(numerator, denominator)
# for 07
min_len = min(len(predictions_07), len(references_07))
predictions_07 = predictions_07[:min_len]
references_07 = references_07[:min_len]
extracted_predictions = [pred['answer'][0] if isinstance(pred, dict) else pred for pred in predictions_07]
numerator_07 = sum(min(len(pred), len(ref)) for pred, ref in zip(extracted_predictions, references_07))
denominator_07 = sum(len(pred) for pred in extracted_predictions)
bleu_score_07 = Fraction(numerator_07, denominator_07)

print("BLEU Score:drop_out 03", bleu_score)
print("BLEU Score:drop_out 07", bleu_score_07)


BLEU Score:drop_out 03 0.6
BLEU Score:drop_out 07 0.8


 # ***`-----------Mobile Bert-------------------`***
 # -----------------Mobile Bert---------------------
 # `--------------------------------------`

In [ ]:
model_args_mobilebert_03 = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "fp16": False,
    "use_cuda": False,
    "num_train_epochs": 3,
    "dropout": 0.3,
}
model_args_mobilebert_07 = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "fp16": False,
    "use_cuda": False,
    "num_train_epochs": 3,
    "dropout": 0.7,
}

# ***For 0.3 Drop Out Rate  ***

In [ ]:

try:
    mobilebert_model_03 = QuestionAnsweringModel("mobilebert", "google/mobilebert-uncased", args=model_args_mobilebert_03, use_cuda=torch.cuda.is_available())
    mobilebert_model_03.train_model(train_examples, eval_data=test_examples, num_samples=5, output_dir="model_output_mobilebert_03/")
except Exception as e:
    print(f"An error occurred during training with dropout rate 0.3: {str(e)}")
try:
    mobilebert_result_03, _ = mobilebert_model_03.eval_model(test_examples)
    print("MobileBERT Results with dropout rate 0.3:", mobilebert_result_03)
    predictions_mobilebert_03 = mobilebert_model_03.predict(test_examples)
    references_mobilebert_03 = [example['context'] for example in test_examples]
except Exception as e:
    print(f"An error occurred during evaluation with dropout rate 0.3: {str(e)}")

Some weights of MobileBertForQuestionAnswering were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Running Evaluation: 100%|██████████| 1/1 [00:20<00:00, 20.09s/it]


MobileBERT Results with dropout rate 0.3: {'correct': 0, 'similar': 25, 'incorrect': 0, 'eval_loss': -70.57221984863281}


Running Prediction: 100%|██████████| 1/1 [00:05<00:00,  5.37s/it]


# ***For 0.7 drop Out Rate ***

In [ ]:
# Train the MobileBERT model with dropout rate 0.7
try:
    mobilebert_model_07 = QuestionAnsweringModel("mobilebert","google/mobilebert-uncased", args=model_args_mobilebert_07, use_cuda=torch.cuda.is_available())
    mobilebert_model_07.train_model(train_examples, eval_data=test_examples, num_samples=5, output_dir="model_output_mobilebert_07/")
except Exception as e:
    print(f"An error occurred during training with dropout rate 0.7: {str(e)}")

# Evaluate the MobileBERT model with dropout rate 0.7
try:
    mobilebert_result_07, _ = mobilebert_model_07.eval_model(test_examples)
    print("MobileBERT Results with dropout rate 0.7:", mobilebert_result_07)
    predictions_mobilebert_07 = mobilebert_model_07.predict(test_examples)
    references_mobilebert_07 = [example['context'] for example in test_examples]
except Exception as e:
    print(f"An error occurred during evaluation with dropout rate 0.7: {str(e)}")

Some weights of MobileBertForQuestionAnswering were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Running Evaluation: 100%|██████████| 1/1 [00:06<00:00,  6.67s/it]


MobileBERT Results with dropout rate 0.7: {'correct': 25, 'similar': 0, 'incorrect': 0, 'eval_loss': 20.118234634399414}


Running Prediction: 100%|██████████| 1/1 [00:04<00:00,  4.98s/it]


# ***Calculate Blue Score For Moile Bert ***

In [ ]:
def extract_text(prediction):
    if isinstance(prediction, dict):
        return prediction['answer'][0]
    else:
        return prediction
min_len_03 = min(len(predictions_mobilebert_03), len(references_mobilebert_03))
predictions_03 = predictions_mobilebert_03[:min_len_03]
references_03 = [ref.split() for ref in references_mobilebert_03[:min_len_03]]
extracted_predictions_03 = [extract_text(pred) for pred in predictions_03]
from fractions import Fraction
numerator = sum(min(len(pred), len(ref)) for pred, ref in zip(extracted_predictions_03, references_03))
denominator = sum(len(pred) for pred in extracted_predictions_03)
numerator = min(1, numerator)
denominator = max(1, denominator)
bleu_score_03 = Fraction(numerator, denominator)


min_len_07 = min(len(predictions_mobilebert_07), len(references_mobilebert_07))
predictions_07 = predictions_mobilebert_07[:min_len_03]
references_07= [ref.split() for ref in references_mobilebert_07[:min_len_03]]
extracted_predictions_07 = [extract_text(pred) for pred in predictions_03]
numerator = sum(min(len(pred), len(ref)) for pred, ref in zip(extracted_predictions_07, references_07))
denominator = sum(len(pred) for pred in extracted_predictions_07)
numerator = min(1, numerator)
denominator = max(1, denominator)
bleu_score_07 = Fraction(numerator, denominator)
print("BLEU Score for dropout rate 0.3:", bleu_score_03)
print("BLEU Score for dropout rate 0.7:", bleu_score_07)


BLEU Score for dropout rate 0.3: 1/50
BLEU Score for dropout rate 0.7: 1/50


 # ***`-----------ROBert-------------------`***
 # -----------------ROBert---------------------
 # `--------------------------------------`

In [11]:

roberta_model = QuestionAnsweringModel('roberta', 'roberta-base', use_cuda=torch.cuda.is_available())
print("Training RoBERTa model with limited data...")
roberta_model.train_model(train_examples, eval_data=test_examples)
roberta_result, model_outputs = roberta_model.eval_model(test_examples)
print("RoBERTa Results with limited data:", roberta_result)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/12 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
add example index and unique id: 100%|██████████| 25/25 [00:00<00:00, 20262.34it/s]


Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

RoBERTa Results with limited data: {'correct': 0, 'similar': 25, 'incorrect': 0, 'eval_loss': -3.3429148197174072}


In [17]:

model_args_03 = {
    "learning_rate": 1e-5,
    "num_train_epochs": 1,
    "train_batch_size": 8,
    "eval_batch_size": 8,
    "max_seq_length": 512,
    "do_lower_case": False,
    "overwrite_output_dir": True,
    "output_dir": "outputs/roberta_dropout_03/",
    "config": {"hidden_dropout_prob": 0.3}
}

model_args_07 = {
    "learning_rate": 1e-5,
    "num_train_epochs": 1,
    "train_batch_size": 8,
    "eval_batch_size": 8,
    "max_seq_length": 512,
    "do_lower_case": False,
    "overwrite_output_dir": True,
    "output_dir": "outputs/roberta_dropout_07/",
    "config": {"hidden_dropout_prob": 0.7}
}

# ***For Drop Out 0.3***

In [11]:
roberta_model_03 = QuestionAnsweringModel('roberta', 'roberta-base', args=model_args_03, use_cuda=torch.cuda.is_available())
limited_train_data = train_examples
print("Training RoBERTa model with dropout rate 0.3 and limited data...")
roberta_model_03.train_model(limited_train_data, eval_data=test_examples)
roberta_result_03, _ = roberta_model_03.eval_model(test_examples)
print("RoBERTa Results with dropout rate 0.3:", roberta_result_03)



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
add example index and unique id: 100%|██████████| 75/75 [00:00<00:00, 256794.12it/s]


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/11 [00:00<?, ?it/s]

add example index and unique id: 100%|██████████| 25/25 [00:00<00:00, 96910.91it/s]


Running Evaluation:   0%|          | 0/4 [00:00<?, ?it/s]

RoBERTa Results with dropout rate 0.3: {'correct': 0, 'similar': 25, 'incorrect': 0, 'eval_loss': -0.06682626344263554}


## ***Training with dropout rate 0.7***

In [19]:
roberta_model_07 = QuestionAnsweringModel('roberta', 'roberta-base', args=model_args_07, use_cuda=torch.cuda.is_available())
print("Training RoBERTa model with dropout rate 0.7 and limited data...")
roberta_model_07.train_model(train_examples, eval_data=test_examples)
roberta_result_07, _ = roberta_model_07.eval_model(test_examples )
print("RoBERTa Results with dropout rate 0.7:", roberta_result_07)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training RoBERTa model with dropout rate 0.7 and limited data...


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
add example index and unique id: 100%|██████████| 75/75 [00:00<00:00, 119246.70it/s]


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/11 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
add example index and unique id: 100%|██████████| 25/25 [00:00<00:00, 209296.61it/s]


Running Evaluation:   0%|          | 0/4 [00:00<?, ?it/s]

RoBERTa Results with dropout rate 0.7: {'correct': 0, 'similar': 25, 'incorrect': 0, 'eval_loss': 0.07012176513671875}


# ***Calculae Blue score***

In [20]:
def extract_text(prediction):
    if isinstance(prediction, dict) and 'answer' in prediction:
        return prediction['answer']
    return prediction
def prepare_bleu_data(predictions, references):
    extracted_predictions = [extract_text(pred) for pred in predictions]
    references = [ref.split() for ref in references]
    extracted_predictions = [pred.split() for pred in extracted_predictions]
    return extracted_predictions, references

In [23]:
predictions_mobilebert_03 = roberta_model_03.predict(test_examples)
references_mobilebert_03 = [example['context'] for example in test_examples]
extracted_predictions_03 = [extract_text(pred) for pred in predictions_mobilebert_03]
numerator = sum(min(len(pred), len(ref)) for pred, ref in zip(extracted_predictions_03, references_mobilebert_03))
denominator = sum(len(pred) for pred in extracted_predictions_03)
numerator = min(1, numerator)
denominator = max(1, denominator)
bleu_score_07 = Fraction(numerator, denominator)
print("BLEU Score for dropout rate 0.3:", bleu_score_07)


BLEU Score for dropout rate 0.3: 1/50


In [22]:
predictions_mobilebert_07 = roberta_model_07.predict(test_examples)
references_mobilebert_07 = [example['context'] for example in test_examples]

extracted_predictions_07 = [extract_text(pred) for pred in predictions_mobilebert_07]
numerator = sum(min(len(pred), len(ref)) for pred, ref in zip(extracted_predictions_07, references_mobilebert_07))
denominator = sum(len(pred) for pred in extracted_predictions_07)
numerator = min(1, numerator)
denominator = max(1, denominator)
bleu_score_07 = Fraction(numerator, denominator)
print("BLEU Score for dropout rate 0.7:", bleu_score_07)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
add example index and unique id: 100%|██████████| 25/25 [00:00<00:00, 196730.96it/s]


Running Prediction:   0%|          | 0/4 [00:00<?, ?it/s]

BLEU Score for dropout rate 0.7: 1/50


# ***--------------------------------------Final Combined Result -------------***

In [29]:
print(f"{'Model':<15}{'Dropout 0.3':<15}{'Dropout 0.7':<15}")
print("="*45)
print(f"{'BERT':<15}{bleu_score_bert_03:<15}{bleu_score_bert_07:<15}")
print(f"{'Mobile BERT':<15}{bleu_score_Monile_bert_03:<15}{bleu_score_bert_07:<15}")
print(f"{'ROBERT':<15}{bleu_score_Robert_03:<15}{bleu_score_Robert_07:<15}")


Model          Dropout 0.3    Dropout 0.7    
BERT           0.6            0.8            
Mobile BERT    0.2            0.8            
ROBERT         0.02           0.02           
